In [0]:
%python
 # File location and type
 file_location = "/mnt/files/Employee.csv"
 file_type = "csv"
 
 # CSV options
 infer_schema = "false"
 first_row_is_header = "true"
 delimiter = ","
 
 # The applied options are for CSV files. For other file types, these will be ignored.
 df = spark.read.format(file_type) \
   .option("inferSchema", infer_schema) \
   .option("header", first_row_is_header) \
   .option("sep", delimiter) \
   .load(file_location)
 
 #display(df)
 
 temp_table_name = "employee_csv"
 df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql
select count(*) from employee_csv

count(1)
1000


In [0]:
%python
# File location and type
file_location = "/mnt/files/upsert_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

# Create a view or table

temp_table_name = "upsert_data_csv"

df.createOrReplaceTempView(temp_table_name)


Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date
1001,Liam,Nassie,Male,105623,07/11/1988,60,Sweden,4,23/10/2014,7,INR,30/11/2020
2,Kristopher,Farr,Female,69531,29/07/1981,13,China,10,28/05/2016,7,AUD,09/09/2018
3,Kellina,Singh,Female,96879,24/07/1984,27,India,12,21/03/2016,9,INR,09/04/2020
4,Tobit,Patel,Female,76070,29/11/1981,44,India,4,12/11/2012,17,INR,04/10/2019
1002,Noah,Bampkin,Male,62228,03/04/1981,54,USA,7,14/07/2013,8,AUD,20/10/2018
1003,Oliver,Caser,Male,106837,27/05/1984,14,China,5,15/10/2014,5,INR,03/04/2020
1004,William,Dooler,Male,77729,03/10/1991,-12,Sweden,17,21/06/2014,9,AUD,24/02/2018
1005,Elijah,Mainson,Male,81590,05/05/1992,60,Sweden,7,14/04/2018,18,INR,29/03/2019
1006,James,Huckle,Male,71206,08/04/1983,-10,Germany,13,15/05/2018,19,AUD,30/12/2020
1007,Benjamin,Tipens,Male,98642,21/03/1986,53,USA,11,31/03/2017,2,INR,22/11/2020


In [0]:
%sql
select count(*) from upsert_data_csv

count(1)
17


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS DATAVOWELDB;
USE DATAVOWELDB

In [0]:
%sql
--LOCATION is where we want to store out data table
--from EMPLOYEE_CSV insert all data which the department_id is not null to this new data table
DROP TABLE IF EXISTS EMPLOYEE_DELTA_TABLE;

CREATE TABLE EMPLOYEE_DELTA_TABLE
USING delta
PARTITIONED BY (DEPARTMENT_ID)
LOCATION "/DATAVOWEL/DELTA/EMPLOYEE_DATA"
AS 
  (
  SELECT * FROM EMPLOYEE_CSV WHERE DEPARTMENT_ID IS NOT NULL
  )


In [0]:
%sql
DESCRIBE DETAIL EMPLOYEE_DELTA_TABLE

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,ba0c0808-7961-4e58-8f08-a094a784fb14,datavoweldb.employee_delta_table,null,dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA,2022-11-03T23:17:08.421+0000,2022-11-03T23:17:15.000+0000,List(Department_id),20,119264,Map(),1,2


In [0]:
%sql
select count(*) from EMPLOYEE_DELTA_TABLE; --988
select count(*) from UPSERT_DATA_CSV;  --17

count(1)
17


In [0]:
%sql
-- IN EMPLOYEE_DELTA_TABLE THERE ARE 988 ROWS
--AFTER DOING MERGE OPERATION 998 ROWS WILL BE THERE AND 7 ROWS WILL BE UPDATED
MERGE INTO EMPLOYEE_DELTA_TABLE                            -- the MERGE instruction is used to perform the upsert
USING upsert_data_csv

ON EMPLOYEE_DELTA_TABLE.employee_id = upsert_data_csv.employee_id -- ON is used to describe the MERGE condition
   
WHEN MATCHED THEN                                           -- WHEN MATCHED describes the update behavior
  UPDATE SET
  EMPLOYEE_DELTA_TABLE.Last_Name = upsert_data_csv.Last_Name   
WHEN NOT MATCHED THEN                                       -- WHEN NOT MATCHED describes the insert behavior
  INSERT (Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date)              
  VALUES (Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date)


In [0]:
%sql
SELECT COUNT(*) FROM EMPLOYEE_DELTA_TABLE

count(1)
998


In [0]:
%sql
DESCRIBE HISTORY EMPLOYEE_DELTA_TABLE

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2022-11-03T23:24:30.000+0000,8645928144849819,magschenywon@gmail.com,MERGE,"Map(predicate -> (spark_catalog.datavoweldb.EMPLOYEE_DELTA_TABLE.`employee_id` = upsert_data_csv.`employee_id`), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(4125592599887365),1103-151030-blo4h8jy,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 274, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 10, numTargetRowsInserted -> 10, numTargetRowsUpdated -> 7, numOutputRows -> 291, numSourceRows -> 17, numTargetFilesRemoved -> 6)",null
0,2022-11-03T23:17:15.000+0000,8645928144849819,magschenywon@gmail.com,CREATE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [""Department_id""], properties -> {})",null,List(4125592599887365),1103-151030-blo4h8jy,null,WriteSerializable,true,"Map(numFiles -> 20, numOutputRows -> 988, numOutputBytes -> 119264)",null


In [0]:
%sql
--check the previous version data when employee_id = 12
SELECT * FROM EMPLOYEE_DELTA_TABLE VERSION AS OF 0 WHERE EMPLOYEE_ID = 12

Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date
12,Kathy,Milstead,Female,121493,17/10/1991,-9,Brazil,8,17/09/2010,8,INR,02/10/2020


In [0]:
%sql
--check the updated version data when employee_id = 12
SELECT * FROM EMPLOYEE_DELTA_TABLE VERSION AS OF 1 WHERE EMPLOYEE_ID = 12

Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date
12,Kathy,Singh,Female,121493,17/10/1991,-9,Brazil,8,17/09/2010,8,INR,02/10/2020


In [0]:
%fs ls /DATAVOWEL/DELTA/EMPLOYEE_DATA/_delta_log

path,name,size
dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/_delta_log/00000000000000000000.crc,00000000000000000000.crc,92
dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/_delta_log/00000000000000000000.json,00000000000000000000.json,23058
dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/_delta_log/00000000000000000001.crc,00000000000000000001.crc,92
dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/_delta_log/00000000000000000001.json,00000000000000000001.json,12351
dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0


In [0]:
%sql
select * from employee_delta_table where employee_id in (21,35,45,47,49)

Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date
21,Sherie,Onele,Female,96185,05/11/1990,-10,China,16,26/01/2014,9,AUD,18/12/2020
35,Gabriel,Ormshaw,Female,120118,06/10/1984,25,USA,16,21/09/2015,19,INR,07/01/2020
45,Devin,Passler,Female,77288,12/10/1982,-10,Japan,16,09/03/2013,17,INR,31/08/2019
47,Bart,Astbery,Male,64429,06/01/1992,52,Japan,16,11/06/2016,1,AUD,26/03/2020
49,Vikky,McCloch,Male,95866,04/04/1989,41,USA,16,13/05/2014,17,INR,21/03/2020


In [0]:
%sql
--update the table by setting first_name to null for above employee_id
UPDATE datavoweldb.EMPLOYEE_DELTA_TABLE SET First_Name = Null where employee_id in (21,35,45,47,49) 

In [0]:
%sql
select * from employee_delta_table where employee_id in (21,35,45,47,49)

Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date
21,null,Onele,Female,96185,05/11/1990,-10,China,16,26/01/2014,9,AUD,18/12/2020
35,null,Ormshaw,Female,120118,06/10/1984,25,USA,16,21/09/2015,19,INR,07/01/2020
45,null,Passler,Female,77288,12/10/1982,-10,Japan,16,09/03/2013,17,INR,31/08/2019
47,null,Astbery,Male,64429,06/01/1992,52,Japan,16,11/06/2016,1,AUD,26/03/2020
49,null,McCloch,Male,95866,04/04/1989,41,USA,16,13/05/2014,17,INR,21/03/2020


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false

key,value
spark.databricks.delta.retentionDurationCheck.enabled,false


In [0]:
%sql
--vacuum is to prevent that user can go back to previous version. Once we set up with this, user won't go back to check any previous versions but the lastes version of data 
VACUUM DATAVOWELDB.employee_delta_table RETAIN 0 HOURS

path
dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA


In [0]:
%sql
select * from employee_delta_table version as of 2 where employee_id in (21,35,45,47,49)

Employee_id,First_Name,Last_Name,Gender,Salary,Date_of_Birth,Age,Country,Department_id,Date_of_Joining,Manager_id,Currency,End_Date
21,null,Onele,Female,96185,05/11/1990,-10,China,16,26/01/2014,9,AUD,18/12/2020
35,null,Ormshaw,Female,120118,06/10/1984,25,USA,16,21/09/2015,19,INR,07/01/2020
45,null,Passler,Female,77288,12/10/1982,-10,Japan,16,09/03/2013,17,INR,31/08/2019
47,null,Astbery,Male,64429,06/01/1992,52,Japan,16,11/06/2016,1,AUD,26/03/2020
49,null,McCloch,Male,95866,04/04/1989,41,USA,16,13/05/2014,17,INR,21/03/2020


In [0]:
%sql
select * from employee_delta_table version as of 1 where employee_id in (21,35,45,47,49)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 375.0 failed 4 times, most recent failure: Lost task 0.3 in stage 375.0 (TID 893, 10.139.64.4, executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/Department_id=16/part-00000-35dfea74-aea7-4eae-be6f-713c58c5f711.c000.snappy.parquet. A file referenced in the transaction log cannot be found. This occurs when data has been manually deleted from the file system rather than using the table `DELETE` statement. For more information, see https://docs.microsoft.com/azure/databricks/delta/delta-intro#frequently-asked-questions
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:349)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:307)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:499)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:54)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:101)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.io.FileNotFoundException: dbfs:/DATAVOWEL/DELTA/EMPLOYEE_DATA/Department_id=16/part-00000-35dfea74-aea7-4eae-be6f-713c58c5f711.c000.snappy.parquet
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$2(DatabricksFileSystemV2.scala:775)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$1(DatabricksFileSystemV2.scala:761)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:455)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:455)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:412)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLo